In [28]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
import math 
import seaborn as sns

# Global variables
print("Configuration")
path_root = "C:\\Users\\annaw\\Documents\\GitHub\\ideal_diet\\src\\"
path_inputs = path_root +"inputs\\"
path_outputs = path_root + "outputs\\"

print("\tSetting parameters")
fao_element = [664,645]

print("\tLoading parameters")
groups = pd.read_excel(path_inputs + "configurations.xlsx", sheet_name='groups')
diets = pd.read_excel(path_inputs + "configurations.xlsx", sheet_name='diets')
years = pd.read_excel(path_inputs + "configurations.xlsx", sheet_name='years')
indicators_country = pd.read_excel(path_inputs + "configurations.xlsx", sheet_name='indicators_country')

print("\tOther things")
years_fao_colnames = ["Y"+str(y.year) for y in years.itertuples(index=True, name='Pandas')]

print("\tSuccessful")

Configuration
	Setting parameters
	Loading parameters
	Other things
	Successful


In [29]:
print("Merging fao data with groups")

print("\tLoading fao data")
data_fao = pd.read_csv(path_inputs + "faostat-food_supply.csv", encoding = "ISO-8859-1")

print("\tMerging fao data with groups")
data_fao_groups = pd.merge(data_fao, groups, left_on = "Item Code",right_on="Item Code",how='inner')

print("\tFiltering columns")
data_fao_colnames = ["ID","Area Code","Area","Item Code","Item_x","Element Code","Element","Unit","Description","Food group"]
data_fao_colnames.extend(years_fao_colnames)
data_fao_groups = data_fao_groups[data_fao_colnames]
data_fao_groups_colnames = ["id","area_code","area","item_code","item","element_code","element","unit","description","food_group"]
data_fao_groups_colnames.extend(years_fao_colnames)
data_fao_groups.columns = data_fao_groups_colnames

print("\tFiltering element " + str(fao_element))
data_fao_groups = data_fao_groups[data_fao_groups.element_code.isin(fao_element)]

print("\tPivoting food groups")
data_fao_groups = pd.pivot_table(data_fao_groups, values=years_fao_colnames, index=['area','food_group'], columns=['element_code'], aggfunc=np.sum,fill_value=0.0)    
data_fao_groups.reset_index(inplace=True)
print(data_fao_groups.columns)
#data_fao_groups.columns = data_fao_groups.columns.droplevel()
data_fao_groups_colnames = ['area','food_group']
for y in years_fao_colnames:
    for e in fao_element:
        data_fao_groups_colnames.append(y + "_" + str(e))
data_fao_groups.columns = data_fao_groups_colnames

print("\tSaving")
data_fao_groups.to_excel(path_outputs +'1-data_fao_groups.xlsx', engine='openpyxl', index = False)

print("\tSuccessful")

Merging fao data with groups
	Loading fao data
	Merging fao data with groups
	Filtering columns
	Filtering element [664, 645]
	Pivoting food groups
MultiIndex(levels=[['Y2013', 'food_group', 'area'], [645, 664, '']],
           codes=[[2, 1, 0, 0], [2, 2, 0, 1]],
           names=[None, 'element_code'])
	Saving
	Successful


In [30]:
print("Merging fao data with diets")

print("\tMerging fao data with diet")
data_fao_diet = pd.merge(data_fao_groups, diets, left_on = "food_group",right_on="food_group",how='inner')

print("\tSaving")
data_fao_diet.to_excel(path_outputs +'2-data_fao_diet.xlsx', engine='openpyxl', index = False)

print("\tSuccessful")

Merging fao data with diets
	Merging fao data with diet
	Saving
	Successful


In [31]:
print("Calculating categories")

print("\tCreating categories fields")
data_fao_cat = data_fao_diet.copy()

for y in years_fao_colnames:
    for e in fao_element:
        data_fao_cat["category_" + y + "_" + str(e)] = 0

print("\tGetting food groups")
food_group = data_fao_cat.food_group.unique()


quintals = pd.DataFrame()

for fg in food_group:
    print("\t\tCalculating food group: " + fg)
    fg_fao = data_fao_cat[data_fao_cat.food_group == fg]
    
    for e in fao_element:
        print("\t\t\tCalculating element: " + str(e))
        diet = "d" + str(e)
        
        for y in years_fao_colnames:
            print("\t\t\tCalculating year: " + y)
            colname =  y + "_" + str(e) 

            # Food groups above
            fg_fao_a = fg_fao[fg_fao.type == "A"] 
            if(fg_fao_a.shape[0]>0):
                # Splitting in two datasets: above or below of diet
                fg_fao_a_above = fg_fao_a[fg_fao_a[colname] >= fg_fao_a[diet]]
                fg_fao_a_below = fg_fao_a[fg_fao_a[colname] < fg_fao_a[diet]]

                # Calculating quintals for below data
                q = fg_fao_a_below[colname].quantile([0.25, 0.5,0.75, 1])
                q["year"] = y.replace("Y","")            
                q["food_group"] = fg
                q["type"] = "A"
                q["element"] = e
                quintals = quintals.append(q, ignore_index=True)

                # Setting category A
                data_fao_cat["category_" + y + "_" + str(e)].iloc[fg_fao_a_above.index] = 5
                fg_fao_a_below["category_" + y + "_" + str(e)] = fg_fao_a_below[colname].apply(lambda x: 1 if x<q[0] else (2 if x<q[1] else (3 if x<q[2] else 4)))
                data_fao_cat["category_" + y + "_" + str(e)].iloc[fg_fao_a_below.index] = fg_fao_a_below["category_" + y + "_" + str(e)]

            # Food groups below
            fg_fao_b = fg_fao[fg_fao.type == "B"] 
            if(fg_fao_b.shape[0]>0):
                # Splitting in two datasets: above or below of diet
                fg_fao_b_below = fg_fao_b[fg_fao_b[colname] <= fg_fao_b[diet]]
                fg_fao_b_above = fg_fao_b[fg_fao_b[colname] > fg_fao_b[diet]]

                # Calculating quintals for below data
                q = fg_fao_b_above[colname].quantile([0.25, 0.5,0.75, 1])
                q["year"] = y.replace("Y","")            
                q["food_group"] = fg
                q["type"] = "B"
                q["element"] = e
                quintals = quintals.append(q, ignore_index=True)

                # Setting category B
                data_fao_cat["category_" + y + "_" + str(e)].iloc[fg_fao_b_below.index] = 5
                fg_fao_b_above["category_" + y + "_" + str(e)] = fg_fao_b_above[colname].apply(lambda x: 1 if x>q[3] else (2 if x>q[2] else (3 if x>q[1] else 4)))
                data_fao_cat["category_" + y + "_" + str(e)].iloc[fg_fao_b_above.index] = fg_fao_b_above["category_" + y + "_" + str(e)]

print("\tSaving")
quintals.to_excel(path_outputs +'3-quintals.xlsx', engine='openpyxl', index = False)  
data_fao_cat.to_excel(path_outputs +'3-data_fao_cat.xlsx', engine='openpyxl', index = False)

print("\tSuccessful")

Calculating categories
	Creating categories fields
	Getting food groups
		Calculating food group: Alcohol
			Calculating element: 664
			Calculating year: Y2013
			Calculating element: 645
			Calculating year: Y2013


C:\Users\annaw\Anaconda3\lib\site-packages\ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


		Calculating food group: Dairy
			Calculating element: 664
			Calculating year: Y2013
			Calculating element: 645
			Calculating year: Y2013


C:\Users\annaw\Anaconda3\lib\site-packages\ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


		Calculating food group: Fats & Oils
			Calculating element: 664
			Calculating year: Y2013
			Calculating element: 645
			Calculating year: Y2013
		Calculating food group: Fruits
			Calculating element: 664
			Calculating year: Y2013
			Calculating element: 645
			Calculating year: Y2013
		Calculating food group: Grains (Starchy staples)
			Calculating element: 664
			Calculating year: Y2013
			Calculating element: 645
			Calculating year: Y2013
		Calculating food group: Legumes, Nuts & Seeds
			Calculating element: 664
			Calculating year: Y2013
			Calculating element: 645
			Calculating year: Y2013
		Calculating food group: Meats
			Calculating element: 664
			Calculating year: Y2013
			Calculating element: 645
			Calculating year: Y2013
		Calculating food group: Miscellaneous
			Calculating element: 664
			Calculating year: Y2013
			Calculating element: 645
			Calculating year: Y2013
		Calculating food group: Spices
			Calculating element: 664
			Calculating year: Y2013
			Calcula

In [32]:
print("Creating matrix for countries analysis")

print("\tPivoting table by country")
data_countries_columns = []
for y in years_fao_colnames:
    for e in fao_element:
        data_countries_columns.append("category_" + y + "_" + str(e))

        

countries_pivot = pd.pivot_table(data_fao_cat, values=data_countries_columns, index=['area'], columns=['type'], aggfunc=np.sum)
countries_pivot_total = countries_pivot.sum(level=0, axis=1)
countries_pivot = countries_pivot.join(countries_pivot_total)
countries_pivot.reset_index(inplace=True)

countries_pivot_colnames = ['area']
countries_pivot_colnames_total = []
for y in years_fao_colnames:
    for e in fao_element:
        for t in ['A','B']:
            countries_pivot_colnames.append(t + "_" + y + "_" + str(e))
        countries_pivot_colnames_total.append("total_" + y + "_" + str(e))

countries_pivot_colnames.extend(countries_pivot_colnames_total)
countries_pivot.columns = countries_pivot_colnames

print("\tSaving")
countries_pivot.to_excel(path_outputs +'4-data_countries.xlsx', engine='openpyxl', index = False)

print("\tSuccessful")

Creating matrix for countries analysis
	Pivoting table by country
	Saving
	Successful


C:\Users\annaw\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:522: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [33]:
print("Countries analysis")

print("\tMerging countries with indicators")
countries_indicators = pd.merge(countries_pivot, indicators_country, left_on = "area",right_on="area",how='inner')

print("\tSaving")
countries_indicators.to_excel(path_outputs +'5-countries_indicators.xlsx', engine='openpyxl', index = False)

# Correlation
print("\tCalculating correlation")
countries_correlation = countries_indicators.corr(method ='pearson')
#countries_correlation.to_excel(path_outputs +'countries_correlation.xlsx', engine='openpyxl', index = False)
def magnify():
    return [dict(selector="th",props=[("font-size", "7pt")]),
            dict(selector="td",props=[('padding', "0em 0em")]),
            dict(selector="th:hover",props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",props=[('max-width', '200px'),('font-size', '12pt')])]

cmap = sns.diverging_palette(5, 250, as_cmap=True)
countries_table_correlation = countries_correlation.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '10pt'})\
    .set_caption("Hover to magify")\
    .set_precision(2)\
    .set_table_styles(magnify())

print("\tSaving")
countries_table_correlation.to_excel(path_outputs +'5-countries_table_correlation.xlsx', engine='openpyxl')

print("\tSuccessful")

Countries analysis
	Merging countries with indicators
	Saving
	Calculating correlation
	Saving
	Successful


In [182]:
print("SSR Analysis")

# Export Quantity = 5911
# Production = 5511
# Import Quantity = 5611
# Stock Variation = 5072
# fao_element = kcal o gperday 
ssr_elements = [5511,5911,5611,5072]
ssr_elements.extend(fao_element)

print("\tCopying fao data")
data_ssr = data_fao.copy()

print("\tMerging fao data with groups")
data_ssr = pd.merge(data_ssr, groups, left_on = "Item Code",right_on="Item Code",how='inner')

print("\tFiltering columns")
data_ssr_colnames = ["ID","Area Code","Area","Item Code","Item_x","Element Code","Element","Unit","Food group"]
data_ssr_colnames.extend(years_fao_colnames)
data_ssr = data_ssr[data_ssr_colnames]
data_ssr_colnames = ["id","area_code","area","item_code","item","element_code","element","unit","food_group"]
data_ssr_colnames.extend(years_fao_colnames)
data_ssr.columns = data_ssr_colnames

print("\tFiltering elements")
data_ssr = data_ssr[data_ssr.element_code.isin(ssr_elements)]
#data_ssr_element = data_ssr[data_ssr.element_code == fao_element ]

print("\tPivoting food groups")
data_ssr_groups = pd.pivot_table(data_ssr, values=years_fao_colnames, index=['area','food_group'], columns=['element_code'], aggfunc=np.sum,fill_value=0.0)    

total = data_ssr_groups["Production"] / (data_ssr_groups["Production"] + data_ssr_groups["Stock Variation"] +data_ssr_groups["Import Quantity"]-data_ssr_groups["Export Quantity"])

print(total)

'''
data_fao_groups.reset_index(inplace=True)
data_fao_groups.columns = data_fao_groups.columns.droplevel()
data_fao_groups_colnames = ['area','food_group']
for y in years_fao_colnames:
    for e in fao_element:
        data_fao_groups_colnames.append(y + "_" + str(e))
data_fao_groups.columns = data_fao_groups_colnames


ssr = pd.DataFrame()

for y in years_fao_colnames:    
    print("\t\tYear: " + y)
    
    ssr_item_pivot = pd.pivot_table(data_ssr, values=y, index=['item','area'], columns=['element'], aggfunc=np.sum,fill_value=0.0)    
    ssr_item_pivot["ssr"] = ssr_item_pivot["Production"] / (ssr_item_pivot["Production"] + ssr_item_pivot["Stock Variation"] +ssr_item_pivot["Import Quantity"]-ssr_item_pivot["Export Quantity"])
    
    # Replacing infinity values with 1
    ssr_item_pivot = ssr_item_pivot.replace(np.inf, 1)
    # Filling empty records
    ssr_item_pivot = ssr_item_pivot.fillna(0)    
    
    print("\t\tSaving items")
    ssr_item_pivot.to_excel(path_outputs +'6-ssr_item_' + y + '.xlsx', engine='openpyxl')
    
    ssr_fg_pivot = pd.pivot_table(data_ssr, values=y, index=['food_group','area'], columns=['element'], aggfunc=np.sum,fill_value=0.0)    
    ssr_fg_pivot["ssr"] = ssr_fg_pivot["Production"] / (ssr_fg_pivot["Production"] + ssr_fg_pivot["Stock Variation"] +ssr_fg_pivot["Import Quantity"]-ssr_fg_pivot["Export Quantity"])
    
    # Replacing infinity values with 1
    ssr_fg_pivot = ssr_fg_pivot.replace(np.inf, 1)
    # Filling empty records
    ssr_fg_pivot = ssr_fg_pivot.fillna(0)   
    
    print(ssr_fg_pivot.columns)
    
    print("\t\tSaving food groups")
    #ssr_fg_pivot.to_excel(path_outputs +'ssr_fg_' + y + '.xlsx', engine='openpyxl')
    #print(ssr_fao_pivot["ssr"])
    
    #countries_pivot = pd.pivot_table(data_fao_cat, values='category' + y, index=['area'], columns=['type'], aggfunc=np.sum)
    #if(data_countries.shape[0] == 0):
    #    data_countries.area = countries_pivot.index.values
    #data_countries["A" + y] = countries_pivot.A.values
    #data_countries["B" + y] = countries_pivot.B.values
    #data_countries["Total" + y] = data_countries["A" + y] + data_countries["B" + y]
        
'''


SSR Analysis
	Copying fao data
	Merging fao data with groups
	Filtering columns
	Filtering elements
	Pivoting food groups


KeyError: 'Production'